### 特征缩放

- 问题：为什么特征的大小尺度很不一样，会导致梯度下降算法变慢？ 为什么把特征缩放到同一个范围内，比如[-1,1],[0,1]会解决梯度下降算法变慢的问题？（从直觉上理解即可）

 1. 线性模型与贡献项

对于多特征线性模型
$$
\hat{y}= w_1x_1+w_2x_2+\dots+w_nx_n+b,
$$

* **每一项 $w_jx_j$** 都在为预测值贡献“加权分数”。
* 如果 $x_j$ 的取值范围比其他特征大得多（如 0 – 3000），那么 **同样大小的权重 $w_j$** 会让这一项产生 **远超其他项** 的数值贡献。

  * 例：面积 $x_1=2000$、卧室数 $x_2=3$。即使 $w_1=w_2=1$，也会有 $w_1x_1 =2000$ 与 $w_2x_2 =3$ 的悬殊差距。
  * 这样会导致预测值几乎只由面积控制，梯度也会优先生长或压缩 $w_1$，其余权重更新“存在感”微弱。


2. 损失函数对“大贡献项”的“惩罚”

以平方损失为例

$$
J(\mathbf{w},b)=\frac1{2m}\sum_{i}( \hat{y}^{(i)}-y^{(i)})^2.
$$

* 如果某个特征项 $w_jx_j$ 过大，$\hat{y}$ 很容易被它**推离**真值 $y$，损失急剧上升。
* 因为损失对误差 $e$ **平方放大**，优化器为了迅速降低 $J$，会对引起大误差的那一项**优先下手**：

  $$
  \frac{\partial J}{\partial w_j}=\frac1m\sum_i e^{(i)}x_j^{(i)}.
  $$

  * 这里的梯度与 $x_j$ **成正比**，数值范围大的特征让梯度也变大，从而导致 **更激进的权重更新**。
* 结果就是：

  * **陡峭方向（大特征）** → 梯度巨大 → 权重调整幅度大，为了抑制贡献而被“拉回”。
  * **平缓方向（小特征）** → 梯度较小 → 权重缓慢更新。
* 在相同学习率 $\alpha$ 下，参数向量会先在陡峭方向来回“过冲—回拉”，同时沿平缓方向慢慢滑动——蜿蜒的轨迹。


3. 为什么缩放能“抚平”这种差距？

    **统一量级后**，所有 $x_j$ 大约落在 $[-1,1]$ 或 $[0,1]$：

    * $\partial J / \partial w_j$ 的幅度不会因为“谁更大”而相差几百倍。
     * 各方向步长相近 → 更新轨迹趋向**径向直线**，收敛速度显著提升。

4. 小结

* **大的特征 $\bigl(x_j \gg x_k\bigr)$**：

  * 同样的权重会导致 **预测贡献极大**，损失函数随之陡峭。
  * 为了快速降损失，优化器会对该权重 **过度惩罚/大幅摆动**——看似把 $w_j$ “限制” 在较小值。
* **梯度 zig‑zag** 本质：

  * 在陡峭（大特征）方向剧烈跳跃，在平缓（小特征）方向慢慢爬坡。
* **解决方式**：
   特征缩放 / 标准化


> **总结**：
> 线性模型里，特征值越大，权重微小变动就能放大误差；损失函数因此“惩罚”它、梯度猛拉它，形成陡峭与平缓方向的极端差异，导致梯度下降之字形前进——缩放后地形变圆，更新才会顺畅。


#### 特征缩放的实现
1. 除以最大值
2. 均值归一化（Mean normalization）
3. Z-score标准化（Z-score normalization）

In [8]:
import numpy as np

# 原始特征矩阵：每行一个样本，列 0=面积，列 1=卧室数
X = np.array([
    [2104, 5],
    [1416, 3],
    [ 852, 2]
], dtype=float)
print("原始 X:\n", X)

# 1. 按最大值缩放
X_max = X / X.max(axis=0)        # 按列除以各自的最大值
print("\n按最大值缩放:\n", X_max)
# 做法：每个特征值除以该列最大值。
# 范围：[0,1] 若所有值 ≥ 0。
# 特点实现最简单，但如果后续出现大于训练集最大值的样本，缩放后值会超出 1。

# 2. 均值归一化
X_mean = (X - X.mean(axis=0)) / (X.max(axis=0) - X.min(axis=0))
print("\nMean normalization:\n", X_mean)
# 做法：先减去列均值，再除以列范围（max-min）
# 范围：大约落在[-1,1]
# 特点：居中到0，且保留相对大小；若分布极端偏斜，效果可能一般。

# 3. Z‑Score 标准化
x_zscore = (X - X.mean(axis=0)) / X.std(axis=0)
print("Z-score 标准化:", x_zscore)
# 做法：减去均值，再除以标准差。
# 范围：均值0，标准差1；数值可正可负，通常在[-3,3]区间
# 特点：对正态分布型（或近似）数据最友好，常用的缩放方法

原始 X:
 [[2.104e+03 5.000e+00]
 [1.416e+03 3.000e+00]
 [8.520e+02 2.000e+00]]

按最大值缩放:
 [[1.         1.        ]
 [0.6730038  0.6       ]
 [0.40494297 0.4       ]]

Mean normalization:
 [[ 0.51650692  0.55555556]
 [-0.03301384 -0.11111111]
 [-0.48349308 -0.44444444]]
Z-score 标准化: [[ 1.26311506  1.33630621]
 [-0.08073519 -0.26726124]
 [-1.18237987 -1.06904497]]


- 什么时候考虑缩放数据？
| 原始特征范围                     | 评估      | 操作建议        |
| -------------------------- | ------- | ----------- |
| $0 \le x_1 \le 3$          | ✅ 范围小   | **不用缩放**    |
| $-2 \le x_2 \le 0.5$       | ✅ 居中且适中 | **不用缩放**    |
| $-100 \le x_3 \le 100$     | ⚠️ 过大   | **需缩放**     |
| $-0.001 \le x_4 \le 0.001$ | ⚠️ 过小   | **需放大/标准化** |
| $98.6 \le x_5 \le 105$     | ⚠️ 偏大   | **需缩放**     |

### 确保梯度下降在**正常工作**

| 目标       | 说明                                                                |
| -------- | ----------------------------------------------------------------- |
| **优化目标** | 最小化代价函数 $J(\mathbf{w}, b)$                                        |
| **学习曲线** | 每一次迭代都应使 $J(\mathbf{w}, b)$ **递减**                                |

> 迭代次数的需求差异很大：30、1 000 甚至 100 000 步都可能出现，取决于学习率和问题规模。



#### 自动收敛判定（伪代码）

```text
设 ε = 1e-3  (收敛阈值)

若  J(t−1) − J(t)  ≤ ε:
    判定为收敛
    -> 已找到使 J 接近全局最小的 (w, b)
```

* **核心思想**：若一次迭代带来的代价下降幅度不超过 ε（例如 0.001），说明模型已基本到达最低点附近，可停止训练。



> **实践提醒**
>
> * 学习率过大：$J$ 可能震荡甚至上升。
> * 学习率过小：$J$ 虽下降，但收敛速度慢，曲线“拖尾”。
> * 建议在训练中实时绘制 **学习曲线**，以及设置 ε‑收敛检验，以便动态调整超参数。


### 选择学习率
1. 如何判断梯度下降是否“出问题”  — 学习率与 Bug 的快速排查

| 现象                         | 可能原因                                                                                | 应对策略                                 |
| -------------------------- | ----------------------------------------------------------------------------------- | ------------------------------------ |
| **锯齿状上下波动**<br/>（但总体仍缓慢下降） |  - 学习率 α 偏大，更新步长过猛，在谷底两侧来回跳 ↔                                                        | ↓ **减小 α**，直至曲线平滑单调降低                |
| **J 指数级上升**                |   1. **符号写反**：<br/> `w = w + α · ∂J` ❌<br/> 应写 `w = w − α · ∂J` ✅<br/>2. **α 远大于最优值** | - 先检查代码是否忘记减号<br/>- 若逻辑无误，显著**调小 α** |
| **J 单调下降，但非常慢**            |  学习率 **过小**，每步只挪一点点                                                                  | ↑ **逐步增大 α**，观察到平滑但更陡的下降即可           |


- 总结：
    1. **先看学习曲线**：若 $J(\mathbf{w},b)$ 每次迭代都应下降；出现震荡或上升即有问题。
    2. **排 Bug**：确认更新公式是 **“减去”** 梯度，而非加。
    3. **调学习率**：

   * 震荡/爆炸 → **减小 α**
   * 缓慢爬坡 → **增大 α**
    4. **经验法则**：找到一个让 $J$ 在每步稳步下降、且下降速度尽可能快的 α；再配合 ε 收敛检测，既高效又安全。


2. 学习率选择简单策略

<img alt="学习率选择" height="600" src="./images/监督学习-回归9.png" width="600"/>
